In [1]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install html5lib

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np
import os
from io import BytesIO
import warnings
warnings.filterwarnings("ignore", "This pattern is interpreted as a regular expression")

# ================================
# CONFIGURATION
# ================================
SIGNED_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Signings Available S3 Summer.html' #Transfers
LOANS_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Loans Available S3 Summer.html' #Loans
UNIVERSAL_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Universe S3 Summer.html'
OUTPUT_PATH = r'C:/Users/Testing Rename nalng/Desktop/Frosinone_Season_3_Summer.xlsx'
LEAGUE_MULTIPLIERS_PATH = r'C:/Users/Testing Rename nalng/Desktop/League Multipliers.xlsx'  # Path to your Excel file

TEXT_COLUMNS = [
    'UID', 'Name', 'Rec', 'EU National', 'Position', 'Pros',
    'Preferred Foot', 'Inf', 'Transfer Value', 'Nat', 'Division', 'Club', 'Personality'
]

PERCENTAGE_COLUMNS = ['Sv %', 'OP-Cr %', 'Hdr %', 'Conv %', 'Pas %', 'Cr C/A', 'Tck R', 'Pens Saved Ratio', 'Pen/R', 'Shot %']

# ================================
# LOAD LEAGUE POWER RATINGS FROM EXCEL
# ================================
def load_league_power(file_path):
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)
        
        # Convert to dictionary with 'League' as key and 'Power Rating' as value
        league_power = dict(zip(df['League'], df['Power Rating']))
        
        # Add default 'Others' value if not present in the file
        if 'Others' not in league_power:
            league_power['Others'] = 5
            
        return league_power
    except Exception as e:
        print(f"Error loading league power ratings: {e}")
        # Return a default dictionary if file can't be loaded
        return {'Others': 5}

# Load the league power ratings
LEAGUE_POWER = load_league_power(LEAGUE_MULTIPLIERS_PATH)

# ================================
# LEAGUE NAME FIXES FOR ENCODING ISSUES
# ================================
LEAGUE_NAME_FIXES = {
    'BrasileirÃ£o AssaÃ­ SÃ©rie A': 'Brasileirão Assaí Série A',
    'Primera FederaciÃ³n Grupo I': 'Primera Federación Grupo I',
    'Liga Profesional de FÃºtbol': 'Liga Profesional de Fútbol',
    'Primera FederaciÃ³n Grupo I': 'Primera Federación Grupo',
    'Primera FederaciÃ³n Grupo III': 'Primera Federación Grupo',
    'Primera FederaciÃ³n Grupo IV': 'Primera Federación Grupo',
    'Primera FederaciÃ³n Grupo V': 'Primera Federación Grupo',
    'Primera FederaciÃ³n Grupo VI': 'Primera Federación Grupo',
    'Primera FederaciÃ³n Grupo VII': 'Primera Federación Grupo',
    'Regionalliga SÃ¼dwest': 'Regionalliga Südwest',
    'Serie C NOW Girone A': 'Serie C NOW',
    'Serie C NOW Girone B': 'Serie C NOW',
    'Serie C NOW Girone C': 'Serie C NOW',
    'Spor Toto SÃ¼per Lig': 'Spor Toto Süper Lig',
    'French National 3 - Group A': 'French National 3',
    'French National 3 - Group B': 'French National 3',
    'French National 3 - Group C': 'French National 3',
    'French National 3 - Group D': 'French National 3',
    'French National 3 - Group E': 'French National 3',
    'French National 3 - Group F': 'French National 3',
    'French National 3 - Group G': 'French National 3',
    'French National 3 - Group H': 'French National 3',
    'French National 3 - Group I': 'French National 3',
    'French National 3 - Group J': 'French National 3',
    'French National 3 - Group K': 'French National 3',
    'French National 3 - Group L': 'French National 3',
    'BrasileirÃ£o Serie B Chevrolet': 'Brasileirão Serie B Chevrolet',
    'Serie D Girone A': 'Serie D',
    'Serie D Girone B': 'Serie D',
    'Serie D Girone C': 'Serie D',
    'Serie D Girone D': 'Serie D',
    'Serie D Girone E': 'Serie D',
    'Serie D Girone F': 'Serie D',
    'Serie D Girone G': 'Serie D',
    'Serie D Girone H': 'Serie D',
    'Serie D Girone I': 'Serie D',
    'Serie D Girone J': 'Serie D',
    'Serie D Girone K': 'Serie D',
    'Regionalliga West': 'Regionalliga',
    'Regionalliga Nord': 'Regionalliga',
    'Regionalliga Südwest': 'Regionalliga',
    'Regionalliga Bayern': 'Regionalliga',
    'Regionalliga Nordost': 'Regionalliga',
    'Russian Second Division A Gold': 'Russian Second Division A',
    'Russian Second Division A Silver': 'Russian Second Division A',  
    'Russian Second Division A Bronze': 'Russian Second Division A',    
    'Russian Second Division B - Group 1': 'Russian Second Division B',
    'Russian Second Division B - Group 2': 'Russian Second Division B',
    'Russian Second Division B - Group 3': 'Russian Second Division B',
    'DR Congo Premier Division A': 'DR Congolese Premier Division',
    'DR Congo Premier Division B': 'DR Congolese Premier Division',

    # Add more fixes as needed
}

def fix_league_names(df):
    """Fix encoding issues in league names in the DataFrame"""
    if 'Division' in df.columns:
        df['Division'] = df['Division'].replace(LEAGUE_NAME_FIXES)
    return df

# ================================
# LOAD HTML FILES
# ================================
if not os.path.exists(SIGNED_HTML_PATH):
    raise FileNotFoundError(f"Signed players HTML not found: {SIGNED_HTML_PATH}")

if not os.path.exists(UNIVERSAL_HTML_PATH):
    raise FileNotFoundError(f"Universal players HTML not found: {UNIVERSAL_HTML_PATH}")

if not os.path.exists(LOANS_HTML_PATH):
    raise FileNotFoundError(f"Loan players HTML not found: {LOANS_HTML_PATH}")

with open(SIGNED_HTML_PATH, 'r', encoding='utf-8') as file:
    signed_tables = pd.read_html(file)
df_signed = signed_tables[0].copy()
df_signed = fix_league_names(df_signed)

with open(UNIVERSAL_HTML_PATH, 'r', encoding='utf-8') as file:
    universal_tables = pd.read_html(file)
df_universal = universal_tables[0].copy()
df_universal = fix_league_names(df_universal)

with open(LOANS_HTML_PATH, 'r', encoding='utf-8') as file:
    loans_tables = pd.read_html(file)
df_loans = loans_tables[0].copy()
df_loans = fix_league_names(df_loans)

In [2]:

# ================================
# UID‑BASED MERGE AND LABEL  ✅ Fixed version
# ================================

def normalize_uid(df):
    df = df.copy()
    # Convert to float → Int64 (nullable integer) → string
    df['UID'] = pd.to_numeric(df['UID'], errors='coerce')  # force numeric, NaNs stay if bad
    df['UID'] = df['UID'].astype('Int64')                  # keep NaNs clean
    df['UID'] = df['UID'].astype(str).str.strip()          # final string
    return df

# 1️⃣ Normalize UIDs
df_signed    = normalize_uid(df_signed)
df_universal = normalize_uid(df_universal)
df_loans = normalize_uid(df_loans)

# 2️⃣ Add the signability flags
df_signed['Signability']    = 'Available for Transfer'
df_universal['Signability'] = 'Not Transferrable'
df_loans['Signability'] = 'Available on Loan'


# 3️⃣ Concatenate signables first, drop duplicate UIDs

df = (
    pd.concat([df_signed, df_loans, df_universal], ignore_index=True)
      .drop_duplicates(subset='UID', keep='first')
      .reset_index(drop=True)
)

# ================================
# CLEANING & CONVERSION
# ================================
def clean_and_convert_data(df):
    def convert_percentage_to_float(df, column):
        if column in df.columns:
            df[column] = (df[column].astype(str)
                          .str.replace('%', '')
                          .replace('-', np.nan)
                          .astype(float))
        return df

    # Convert minutes
    if 'Mins' in df.columns:
        df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
        df = df[df['Mins'] >= 900].copy()

    # Convert percentage stats
    for col in PERCENTAGE_COLUMNS:
        df = convert_percentage_to_float(df, col)

    # Handle 'Dist/90' - e.g., "7.3mi"
    if 'Dist/90' in df.columns:
        df['Dist/90'] = (
            df['Dist/90']
            .astype(str)
            .str.extract(r'([\d.]+)')[0]
        )

        # Drop only if the string was completely missing
        df['Dist/90'] = pd.to_numeric(df['Dist/90'], errors='coerce')

    print("Dist/90 (after cleaning):")
    print(df['Dist/90'].describe())
    print(df['Dist/90'].dropna().head(10))






    

    # Convert all other numerical columns (except text & signability)
    for col in df.columns:
        if col not in TEXT_COLUMNS + ['Signability']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle 'Dist/90' - e.g., "7.3mi"
    if 'Dist/90' in df.columns:
        df['Dist/90'] = (
            df['Dist/90']
            .astype(str)
            .str.extract(r'([\d.]+)')[0]
        )

        # Drop only if the string was completely missing
        df['Dist/90'] = pd.to_numeric(df['Dist/90'], errors='coerce')

    print("Dist/90 (after cleaning):")
    print(df['Dist/90'].describe())
    print(df['Dist/90'].dropna().head(10))


    # Add league strength multiplier
    def get_league_multiplier(division):
        return LEAGUE_POWER.get(division, LEAGUE_POWER['Others']) / 100.0

    df['League Multiplier'] = df['Division'].apply(get_league_multiplier)
    return df

# ✅ Apply cleaning
df = clean_and_convert_data(df)

# ================================
# CONVERT RAW STATS TO PER 90
# ================================
RAW_STATS_TO_PER90 = {
    "Yel": "Yellow/90",
    "Red": "Red/90",
    "Fls": "FoulsMade/90",
    "FA": "FoulsAgainst/90",
    "Off": "Offsides/90",
    "Gl Mst": "Gl Mst/90",
    "Goals Outside Box": "Goals Outside Box/90",
    "FK Shots": "FKShots/90"
}

for raw_stat, per90_stat in RAW_STATS_TO_PER90.items():
    if raw_stat in df.columns:
        df[raw_stat] = pd.to_numeric(df[raw_stat], errors='coerce')
        df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
        df[per90_stat] = (df[raw_stat] / (df['Mins'] / 90)).fillna(0)
    else:
        print(f"Warning: Raw stat '{raw_stat}' not found in DataFrame")

print(df['Dist/90'].dropna().unique()[:50])
print(df['Dist/90'].dtype)


Dist/90 (after cleaning):
count    20073.000000
mean         4.861515
std          3.225443
min          0.000000
25%          0.300000
50%          6.700000
75%          7.500000
max          9.000000
Name: Dist/90, dtype: float64
0    6.8
1    7.4
2    6.9
3    6.6
4    6.6
5    6.3
6    6.6
7    6.6
8    6.6
9    6.7
Name: Dist/90, dtype: float64
Dist/90 (after cleaning):
count    20073.000000
mean         4.861515
std          3.225443
min          0.000000
25%          0.300000
50%          6.700000
75%          7.500000
max          9.000000
Name: Dist/90, dtype: float64
0    6.8
1    7.4
2    6.9
3    6.6
4    6.6
5    6.3
6    6.6
7    6.6
8    6.6
9    6.7
Name: Dist/90, dtype: float64
[6.8 7.4 6.9 6.6 6.3 6.7 6.4 5.9 7.  7.6 7.5 5.7 7.1 4.9 6.1 7.3 7.2 6.5
 7.7 6.2 7.8 5.1 8.4 8.1 6.  7.9 8.  5.8 5.3 5.6 5.5 8.3 4.3 5.4 5.  8.2
 5.2 3.4 4.7 4.8 4.2 2.9 4.6 3.1 3.5 2.4 4.1 3.9 4.4 1.4]
float64


In [3]:
df["Expires"] = df["Expires"].astype("string")
print(df["Expires"].dtype)

string


In [4]:
# Show overall info
print("==== DataFrame Info ====")
print(df.info())

print("\n==== Column Summary ====")
for col in df.columns:
    print(f"\n📌 Column: {col}")
    print(f"Type: {df[col].dtype}")
    print(f"Missing: {df[col].isna().sum()} / {len(df)}")
    
    if df[col].dtype == 'object':
        unique_vals = df[col].dropna().unique()
        print(f"Unique Values: {len(unique_vals)}")
        print("Sample:", unique_vals[:10])
    
    elif pd.api.types.is_numeric_dtype(df[col]):
        print(df[col].describe())

    else:
        print("Sample:", df[col].dropna().head(5).tolist())

==== DataFrame Info ====
<class 'pandas.core.frame.DataFrame'>
Index: 20073 entries, 0 to 20073
Data columns (total 88 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EU National           20073 non-null  object 
 1   Name                  20073 non-null  object 
 2   UID                   20073 non-null  object 
 3   Mins                  20073 non-null  float64
 4   Position              20073 non-null  object 
 5   Shot %                18250 non-null  float64
 6   Personality           1316 non-null   object 
 7   Age                   20073 non-null  float64
 8   Preferred Foot        20073 non-null  object 
 9   Rec                   20073 non-null  object 
 10  Club                  20073 non-null  object 
 11  Transfer Value        20073 non-null  object 
 12  Sv %                  1809 non-null   float64
 13  OP-Cr %               13863 non-null  float64
 14  Off                   20073 non-null  float64
 15 

In [5]:
if df['Dist/90'].isna().any():
    print("⚠️ Warning: Missing or zero values found in Dist/90")

In [6]:
# ================================
# DATA PREPROCESSING
# ================================
if 'Mins' in df.columns:
    df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
    df = df[df['Mins'] >= 900].copy()

# Create composite metrics before scaling
df['Intensity'] = df["Sprints/90"] / df["Dist/90"].replace(0, 1)  # Avoid division by zero
df['NetPoss'] = df["Poss Won/90"] - df["Poss Lost/90"]
# For attacking players:
df['ChanceCreation'] = 0.20 * df['Ch C/90'] + 0.80* df['xA/90']
# For defenders:
df['AerialDominance'] = (df['Hdrs W/90'] * df['Hdr %']) / 100

# Define numeric columns to scale (excluding text columns and Signability)
TEXT_COLUMNS = ['EU National', 'Name', 'UID', 'Position', 'Personality', 'Preferred Foot', 
                'Rec', 'Club', 'Transfer Value', 'Division', 'Nat', 'Inf', 'Signability','Expires']
                
numeric_columns = [col for col in df.columns 
                   if col not in TEXT_COLUMNS 
                   and pd.api.types.is_numeric_dtype(df[col])]

# Scale numeric columns (0-1)
for col in numeric_columns:
    if col == 'Age':
        continue  # Skip scaling for Age column
    if df[col].nunique() > 1:  # Only scale if there's variation
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = (df[col] - min_val) / (max_val - min_val)
    else:
        df[col] = 0.5  # Default value for constant columns

# Add league multiplier (no scaling needed)
def get_league_multiplier(division):
    return LEAGUE_POWER.get(division, LEAGUE_POWER['Others']) / 100.0

df['League Multiplier'] = df['Division'].apply(get_league_multiplier)

# ================================
# UPDATED ARCHETYPE FORMULAS (SCALED)
# ================================
ARCHETYPE_FORMULAS = {
    "Sweeper Keeper": {
        "filter": df['Position'].str.contains("GK", case=False, na=False),
        "formula": lambda d: (
            0.80 * d["xGP/90"] * d["Sv %"] +
            0.10 * (1 - d["Gl Mst/90"]) +
            0.10 * d["Cln/90"]
        ),
        "label": "SK Rating"
    },
    
    "Central Defender": {
        "filter": df['Position'].str.contains(r"^D\s*\(([RLC]*C[RLC]*)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * ((d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["AerialDominance"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"])) / 7) +  # Average of 7 metrics
            0.10 * (1 - d["Gl Mst/90"]) +
            0.10 * (d["Pas %"] * (d["Pr passes/90"] + d["Drb/90"]) / 2)
        ),
        "label": "CD Rating"
    },
    
    "Fullback": {
        "filter": df['Position'].str.contains(r"^(D|WB)\s*\((R|L|RL|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["xA/90"] +
            0.20 * d["Intensity"]
        ),
        "label": "FB Rating"
    },
    
    "Defensive Midfielder": {
        "filter": df['Position'].str.contains(r"^DM$", regex=True, na=False),
        "formula": lambda d: (
            0.80 * ((d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["AerialDominance"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"])) / 7) +  # Average of 7 metrics
            0.10 * (1 - d["Gl Mst/90"]) +
            0.10 * d["Pas %"] * d["Pr passes/90"] 

        ),
        "label": "DM Rating"
    },
    
    "Winger": {
        "filter": df['Position'].str.contains(r"^AM\s*\((L|R|RL|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["NP-xG/90"] + 
            0.20 * d["xA/90"]
            
        ),
        "label": "W Rating"
    },

    "Central Midfielder": {
        "filter": df['Position'].str.contains(r"^M\s*\((C|RC|LC|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * (d["NP-xG/90"] + d["xA/90"]) / 2 +
            0.20 * ((d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["AerialDominance"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"])) / 7) +  # Average of 7 metrics

        ),
        "label": "CM Rating"
    },
    
    "Striker": {
        "filter": df['Position'].str.contains(r"ST", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["NP-xG/90"] * d["Conv %"] +
            0.20 * d["AerialDominance"]
        ),
        "label": "ST Rating"
    }
}



# ================================
# OUTPUT TO EXCEL WITH SUMMARY SHEET
# ================================
output_excel = BytesIO()

with pd.ExcelWriter(output_excel, engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
    workbook = writer.book
    unicode_format = workbook.add_format({'font_name': 'Arial Unicode MS', 'valign': 'vcenter'})
    
    # First create the summary sheet at the beginning
    worksheet = workbook.add_worksheet('Player Archetype Summary')
    writer.sheets['Player Archetype Summary'] = worksheet
    
    # Dictionary to store archetype data for summary
    summary_data = []
    
    # Process all archetypes
    for role, config in ARCHETYPE_FORMULAS.items():
        role_df = df[config["filter"]].copy()
        if role_df.empty:
            continue

        # Calculate ratings
        role_df[config["label"]] = config["formula"](role_df)
        adjusted_label = f"Adjusted {config['label']}"
        role_df[adjusted_label] = role_df[config["label"]] * role_df['League Multiplier']
        
        # Calculate percentile for each player within this archetype
        role_df['Percentile'] = role_df[adjusted_label].rank(pct=True)
        
        # Store data for summary
        top_players = role_df[role_df['Percentile'] > 0.95][['UID', 'Name', 'Percentile']].copy()
        top_players['Archetype'] = role
        summary_data.append(top_players)
        
        # Write archetype sheet
        role_df['Ranking'] = role_df[adjusted_label].rank(method='min', ascending=False)
        
        output_cols = [
            'UID', 'Name', 'Age', 'Personality', 'Signability', 'EU National', 'Position', 'Preferred Foot',
            'Transfer Value', 'Nat', 'Division', 'Club',
            config["label"], adjusted_label, 'Percentile', 'Ranking', 'League Multiplier', 'Expires'
        ]
        
        result_df = role_df[output_cols].copy().sort_values(by=adjusted_label, ascending=False)
        result_df.to_excel(writer, sheet_name=role, index=False)
        
        # Format archetype sheet
        archetype_sheet = writer.sheets[role]
        for i, col in enumerate(output_cols):
            archetype_sheet.set_column(i, i, 20 if col != 'Name' else 30, unicode_format)
        
        # Format percentile as percentage
        percent_format = workbook.add_format({'num_format': '0.0%'})
        percentile_col_idx = output_cols.index('Percentile')
        archetype_sheet.set_column(percentile_col_idx, percentile_col_idx, 12, percent_format)
    
    # ================================
    # BUILD SUMMARY SHEET DATA
    # ================================
    if summary_data:
        all_archetypes = pd.concat(summary_data).sort_values(['UID', 'Percentile'], ascending=[True, False])
        
        # Group and format archetypes
        def format_archetypes(group):
            return ", ".join(f"{row['Archetype']} ({row['Percentile']:.1%})" for _, row in group.iterrows())
        
        player_summary = (
            all_archetypes.groupby(['UID', 'Name'])
            .apply(format_archetypes)
            .reset_index(name='Top Archetypes (>95%)')
        )
        
        # Add additional info
        player_summary = player_summary.merge(
            df[['UID', 'Position', 'Age', 'Nat', 'Club', 'Division', 'Personality', 'Signability', 'Transfer Value']],
            on='UID', how='left'
        )
        
        # Final columns and sorting
        player_summary = player_summary[[
            'UID', 'Name', 'Position', 'Club', 'Division', 
            'Signability', 'Transfer Value', 'Age', 'Nat', 'Personality', 'Top Archetypes (>95%)'
        ]]
        player_summary['Archetype Count'] = player_summary['Top Archetypes (>95%)'].str.count(',') + 1
        player_summary = player_summary.sort_values(['Archetype Count', 'Name'], ascending=[False, True])
        
        # Write to summary sheet
        player_summary.to_excel(writer, sheet_name='Player Archetype Summary', index=False)
        
        # Format summary sheet
        summary_sheet = writer.sheets['Player Archetype Summary']
        summary_sheet.set_column('A:A', 15)  # UID
        summary_sheet.set_column('B:B', 30)  # Name
        summary_sheet.set_column('C:C', 15)  # Position
        summary_sheet.set_column('D:D', 25)  # Club
        summary_sheet.set_column('E:E', 25)  # Division
        summary_sheet.set_column('F:F', 15)  # Signability
        summary_sheet.set_column('G:G', 15)  # Transfer Value
        summary_sheet.set_column('H:H', 60)  # Age
        summary_sheet.set_column('I:I', 60)  # Nationality
        summary_sheet.set_column('J:J', 60)  # Personality
        summary_sheet.set_column('K:K', 60)  # Top Archetypes

# ================================
# SAVE FILE
# ================================
try:
    with open(OUTPUT_PATH, 'wb') as output_file:
        output_file.write(output_excel.getbuffer())
    print(f"\n✅ Success! Output file saved:\n{OUTPUT_PATH}")
    print(f"Includes {len(ARCHETYPE_FORMULAS)} archetype sheets + summary sheet")
except Exception as e:
    print(f"\n❌ Error saving file: {e}")
    print("Please check directory permissions or disk space.")


C:\Users\Testing Rename nalng\AppData\Local\Temp\ipykernel_10204\123720661.py:190: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(format_archetypes)



✅ Success! Output file saved:
C:/Users/Testing Rename nalng/Desktop/Frosinone_Season_3_Summer.xlsx
Includes 7 archetype sheets + summary sheet


In [7]:
# Test each archetype with dummy perfect player
test_player = {col: 1 for col in numeric_columns}
for name, archetype in ARCHETYPE_FORMULAS.items():
    print(f"{name}: {archetype['formula'](test_player)}")

Sweeper Keeper: 0.9
Central Defender: 0.9
Fullback: 1.0
Defensive Midfielder: 1.0
Winger: 1.0
Attacking Midfielder: 1.0
Striker: 1.0
